In [ ]:
!git clone https://github.com/mshumer/verifiers.git

In [ ]:
!cd verifiers && pwd && pip install -e .

In [ ]:
!wget https://raw.githubusercontent.com/huggingface/trl/refs/heads/main/examples/accelerate_configs/deepspeed_zero3.yaml

In [ ]:
# script = """# script.py
# import verifiers as vf
# from trl import GRPOTrainer

# model_name = "Qwen/Qwen2.5-1.5B-Instruct"
# model, tokenizer = vf.get_model_and_tokenizer(model_name)

# vf_env = vf.DoubleCheckEnv(dataset="gsm8k")
# trainer = GRPOTrainer(
#     model=model,
#     processing_class=tokenizer,
#     env=vf_env,
#     reward_funcs=[vf.llm_judge_reward_func],  # Using LLM-as-Judge for reward
#     args=vf.get_default_grpo_config(run_name="doublecheck", num_gpus=1),
#     train_dataset=vf_env.get_dataset(),
# )
# trainer.train()"""

script="""
from datasets import Dataset
from verifiers.utils.data_utils import format_prompt

# Suppose you have your list of 1000 prompt strings:
my_prompts = [
    # Basic greetings and personal questions
    "Hey, how are you?",
    "What's your name?",
    "Tell me a joke.",
    "What's the meaning of life?",
    "How do you spend your free time?",
    "What's your favorite book?",
    "Where are you from?",
    "What makes you happy?",
    "How was your weekend?",
    "What's your dream vacation?",
    
    # Knowledge and learning
    "Explain quantum physics simply.",
    "What's the most interesting fact you know?",
    "How does the internet work?",
    "What's your take on artificial intelligence?",
    "Can you explain blockchain?",
    "What's the future of space exploration?",
    "How do airplanes stay in the air?",
    "What causes northern lights?",
    "How does photosynthesis work?",
    "What's the theory of evolution?",
    
    # Philosophy and deep thoughts
    "Is time travel possible?",
    "What is consciousness?",
    "Do parallel universes exist?",
    "What happens after death?",
    "Is there free will?",
    "What is reality?",
    "Are we living in a simulation?",
    "What is the nature of happiness?",
    "Can machines think?",
    "What defines human nature?",
    
    # Creative and hypothetical
    "If you could have any superpower, what would it be?",
    "What would you do if you won the lottery?",
    "Create a story about dragons.",
    "Describe your perfect day.",
    "If you could meet anyone from history, who would it be?",
    "What would life be like on Mars?",
    "Design your dream house.",
    "Invent a new holiday.",
    "Create a new sport.",
    "What would you name your spaceship?",
    
    # Problem solving
    "How would you solve world hunger?",
    "What's the best way to learn a new language?",
    "How can we protect the environment?",
    "What's the solution to traffic congestion?",
    "How can we improve education?",
    "What's the best way to stay motivated?",
    "How can we reduce stress?",
    "What's the secret to success?",
    "How can we build better communities?",
    "What's the best way to manage time?",
    
    # Cultural and social
    "What's your favorite movie and why?",
    "How has social media changed society?",
    "What's the role of art in society?",
    "How do different cultures celebrate life?",
    "What makes a good friend?",
    "How has technology changed relationships?",
    "What defines a culture?",
    "How do fashion trends start?",
    "What makes music universal?",
    "How do languages evolve?",
    
    # Science and technology
    "What's the next big technological breakthrough?",
    "How do computers learn?",
    "What's the future of renewable energy?",
    "How do black holes work?",
    "What's the most important scientific discovery?",
    "How does memory work?",
    "What's the future of medicine?",
    "How do we measure the universe?",
    "What's the smallest thing in existence?",
    "How do dreams work?",
    
    # Personal development
    "What's your biggest achievement?",
    "How do you handle failure?",
    "What's your life philosophy?",
    "What motivates you?",
    "How do you define success?",
    "What's your biggest fear?",
    "How do you stay creative?",
    "What's your learning style?",
    "How do you make decisions?",
    "What's your greatest strength?",
    
    # Future and predictions
    "What will cities look like in 100 years?",
    "How will we communicate in the future?",
    "What jobs will exist in 50 years?",
    "What will transportation look like?",
    "How will climate change affect us?",
    "What will education be like in the future?",
    "How will we entertain ourselves?",
    "What will homes look like?",
    "How will we eat in the future?",
    "What will society value most?",
    
    # Ethics and morality
    "What makes something right or wrong?",
    "Should AI have rights?",
    "Is privacy a human right?",
    "What's the value of truth?",
    "How do we balance individual and collective needs?",
    "What are the limits of freedom?",
    "Should we edit human genes?",
    "What do we owe future generations?",
    "Is immortality desirable?",
    "How do we define justice?",
    
    # Miscellaneous
    "What's your favorite food?",
    "Do you believe in aliens?",
    "What's the most beautiful place you've seen?",
    "What sound do you love most?",
    "What's your favorite season?",
    "What animal would you be?",
    "What's your ideal temperature?",
    "What's your favorite word?",
    "What makes you laugh?",
    "What's your favorite time of day?"
]

# Optionally, define a system prompt and few-shot examples if desired:
system_prompt = '''
Respond in the following format, using careful step-by-step reasoning.

<reasoning>
...
</reasoning>
<answer>
...
</answer>
'''
few_shot = []  # or a list of few-shot message dicts

# Convert your list of prompts into the expected format using format_prompt:
formatted_prompts = [format_prompt(prompt, system_prompt=system_prompt, few_shot=few_shot) for prompt in my_prompts]

# Create a dataset; if you don't have ground-truth answers, you can set them to None:
data_dict = {
    "prompt": formatted_prompts,
    "answer": [None] * len(my_prompts)
}
custom_dataset = Dataset.from_dict(data_dict)

# Then, when setting up your training, pass this dataset:
from trl import GRPOTrainer
import verifiers as vf

model_name = "Qwen/Qwen2.5-1.5B-Instruct"
model, tokenizer = vf.get_model_and_tokenizer(model_name)

# For instance, using a SimpleEnv, you might override its get_dataset method:
class CustomEnv(vf.SimpleEnv):
    def get_dataset(self, **kwargs):
        return custom_dataset

    def get_rubric(self, **kwargs):
        # Return a list of reward functions.
        # In this example, we use the llm_judge_reward_func.
        return [vf.llm_judge_reward_func]

vf_env = CustomEnv(system_prompt=system_prompt, few_shot=few_shot)

training_args = vf.get_default_grpo_config(run_name="custom_dataset_run", num_gpus=1)
trainer = GRPOTrainer(
    model=model,
    processing_class=tokenizer,
    reward_funcs=[vf.llm_judge_reward_func],  # Using LLM-as-Judge for reward
    env=vf_env,
    args=training_args,
    train_dataset=custom_dataset,
)
trainer.train()
"""


# Save script to script.py
with open("/content/verifiers/script.py", "w") as f:
    f.write(script)

In [ ]:
!accelerate launch --config_file /content/deepspeed_zero3.yaml --num_processes 1 /content/verifiers/script.py